In [91]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [92]:
from string import translate, maketrans

In [93]:
home = expanduser("~")
annee="17"
def getSerie(nomKalaba):
    return home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/%s-%s/"%(annee,nomKalaba)

prevSerie=getSerie("K2")
nextSerie=getSerie("K1")


In [94]:
with open(prevSerie+"Stems.yaml", 'r') as stream:
    prevStems=yaml.load(stream)
with open(nextSerie+"Stems.yaml", 'r') as stream:
    nextStems=yaml.load(stream)

In [95]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [96]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(nextStems)

In [97]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [98]:
with open(prevSerie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)

In [99]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [100]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in "iae":
            cibles.append(p)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=phonology["apophonies"][result[cible]]
        return "".join(result)
    else:
        return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [101]:
operations=[devoiserFinale,apophonie,voiserVCV,metathese,changerLongueur]

In [102]:
def modifierRadical(radical):
    nOperations=rd.choice([0,1,2,2,3,3,3,4,5])
    while nOperations>0:
        operation=rd.choice(operations)
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    return radical

In [103]:
racineChanges={}
for mot in prevLexique:
    if mot in nextLexique:
#        print mot, ":", prevLexique[mot], ">",nextLexique[mot]
        racineChanges[nextLexique[mot]]=modifierRadical(prevLexique[mot])
    else:
        print "***"
        print mot,"n'est pas dans le nouveau lexique"
        print "***"

***
villageois.HYPER, villagEOIs, villagEOIS n'est pas dans le nouveau lexique
***
***
loup, loups n'est pas dans le nouveau lexique
***
***
villageois, villageoiS n'est pas dans le nouveau lexique
***
***
loup.HYPER, lOUP, lOUPs n'est pas dans le nouveau lexique
***
***
enfant.HYPER, enfANT, enfANTs n'est pas dans le nouveau lexique
***
***
garçon, garçons, enfant, enfants n'est pas dans le nouveau lexique
***


In [120]:
modifierRadical("Tonif")

'sTonuf'

In [105]:
racineChanges

{'SeSer': 'gunov',
 'Senna': 'moZuni',
 'Setane': 'jodun',
 'Sop': 'Subaro',
 'Suf': 'mukutu',
 'Zemala': 'zuZoZi',
 'abita': 'moDona',
 'ad': 'wubuN',
 'allo': 'guSoro',
 'amne': 'noDoNu',
 'amo': 'folov',
 'amplet': 'tujopi',
 'anpal': 'tuDob',
 'apwij': 'subuSo',
 'ar': 'ruzu',
 'asaj': 'Zobulu',
 'astra': 'gaZolu',
 'atame': 'sutogu',
 'atlas': 'Todup',
 'atros': 'lozofe',
 'avans': 'Suzora',
 'banan': 'tubugu',
 'bibipo': 'komof',
 'boske': 'ronuk',
 'braje': 'kofut',
 'brav': 'Zuvop',
 'buf': 'muZuf',
 'buro': 'sulok',
 'bwaje': 'Zukovu',
 'deSik': 'muluko',
 'debark': 'bumuw',
 'deka': 'tuSuS',
 'derul': 'fuDedu',
 'dollore': 'ludure',
 'eSap': 'noDaf',
 'elans': 'tTuZof',
 'estomp': 'lDuvuN',
 'estrop': 'Novut',
 'frapo': 'zozuga',
 'friro': 'Zozut',
 'furo': 'ruvudu',
 'fuzie': 'kozuTS',
 'garuva': 'poNuf',
 'gren': 'toDun',
 'grife': 'rugol',
 'gur': 'susok',
 'intro': 'Tofut',
 'jurta': 'dulum',
 'kadot': 'boZum',
 'kajnina': 'kuvod',
 'kalebe': 'lugop',
 'karne': 'nozuZ',
 

In [106]:
newStems=replace_keys(nextStems,racineChanges)

In [107]:
with open(nextSerie+'flowStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [108]:
with open(nextSerie+"flowStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [109]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  Novut: [blessé, blessés, blessée, blessées]
  Zupov: [trois]
  Zuvop: [courageux, courageuse, courageuses]
  duwoz: [bas, basse, basses]
  fodoT: [furieux, furieuse, furieuses]
  goduf: [effrayé, effrayés, effrayée, effrayées]
  gojoT: [gros, grosse, grosses]
  juzoT: [grand, grande, grands, grandes]
  lozofe: [terrible, terribles]
  luDoto: [maigre, maigres]
  moduk: [blanc, blanche, blancs, blanches]
  noduzu: [rouge, rouges]
  noput: [quatre]
  nubodu: [différent, différents, différente, différentes]
  nuvod: [petit, petite, petits, petites]
  puvub: [suivant, suivants, suivante, suivantes]
  seSuk: [profond, profonds, profonde, profondes]
  tubugu: [jaune, jaunes]
  zogoZ: [noir, noire, noirs, noires]
DET:
  k: [DEF, le, la, les, l]
  l: [IND, un, une, des, dE]
  vu: [DEM, ce, cet, cette, ces]
NOM:
  F:
    NomuT: [autruche, autruches]
    doZub: [poisson, poissons]
    dolud: [ombre, ombres]
    dulum: [maison, maisons]
    forut: [mort, morts]
    gaZolu: [lune, lunes]
   

In [110]:
with open(nextSerie+'newStems.yaml', 'w') as output:
    output.write(yamlText)